# The Project #
1. This is a project with minimal scaffolding. Expect to use the the discussion forums to gain insights! It’s not cheating to ask others for opinions or perspectives!
2. Be inquisitive, try out new things.
3. Use the previous modules for insights into how to complete the functions! You'll have to combine Pillow, OpenCV, and Pytesseract
4. There are hints provided in Coursera, feel free to explore the hints if needed. Each hint provide progressively more details on how to solve the issue. This project is intended to be comprehensive and difficult if you do it without the hints.

### The Assignment ###
Take a [ZIP file](https://en.wikipedia.org/wiki/Zip_(file_format)) of images and process them, using a [library built into python](https://docs.python.org/3/library/zipfile.html) that you need to learn how to use. A ZIP file takes several different files and compresses them, thus saving space, into one single file. The files in the ZIP file we provide are newspaper images (like you saw in week 3). Your task is to write python code which allows one to search through the images looking for the occurrences of keywords and faces. E.g. if you search for "pizza" it will return a contact sheet of all of the faces which were located on the newspaper page which mentions "pizza". This will test your ability to learn a new ([library](https://docs.python.org/3/library/zipfile.html)), your ability to use OpenCV to detect faces, your ability to use tesseract to do optical character recognition, and your ability to use PIL to composite images together into contact sheets.

Each page of the newspapers is saved as a single PNG image in a file called [images.zip](./readonly/images.zip). These newspapers are in english, and contain a variety of stories, advertisements and images. Note: This file is fairly large (~200 MB) and may take some time to work with, I would encourage you to use [small_img.zip](./readonly/small_img.zip) for testing.

Here's an example of the output expected. Using the [small_img.zip](./readonly/small_img.zip) file, if I search for the string "Christopher" I should see the following image:
![Christopher Search](./readonly/small_project.png)
If I were to use the [images.zip](./readonly/images.zip) file and search for "Mark" I should see the following image (note that there are times when there are no faces on a page, but a word is found!):
![Mark Search](./readonly/large_project.png)

Note: That big file can take some time to process - for me it took nearly ten minutes! Use the small one for testing.

In [46]:
from zipfile import ZipFile 
from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

database = {}

def init(fileName):
    database = {}
    with ZipFile(fileName, 'r') as archive:
        for entry in archive.infolist():
            with archive.open(entry) as file:
                img = Image.open(file)
                gray_img= img.convert("L")

                text = pytesseract.image_to_string(gray_img) 

                np_img = np.array(gray_img)
                faces = face_cascade.detectMultiScale(np_img, 1.35)
                database[entry.filename] = {'image': img, 'text': text, 'faces': faces}
                
def createContactSheet(img, faces):
    SIZE = 100
    MAX_COL = 5
    MAX_ROW = (len(faces) - 1 + SIZE) // SIZE
    contact_sheet = Image.new(img.mode, (SIZE * MAX_COL, SIZE * MAX_ROW))
    
    px=0
    py=0
    
    np_img = np.array(img)     
    for x,y,w,h in faces:
        f = np_img[y:y+h, x:x+w]
        face = Image.fromarray(f)
        face.thumbnail((SIZE, SIZE)) 
        
        contact_sheet.paste(face, (px, py))
        if px + SIZE == contact_sheet.width:
            px=0
            py=py+SIZE
        else:
            px=px+SIZE
    return contact_sheet

def search(word):
    for (fileName, info) in database.items():
        print(info)
        print(info["text"])
        if word in info["text"]:
            print("Result found in {}", fileName)
            if len(info["faces"]) > 0:
                contact_sheet = createContactSheet(info["image"], info["faces"])
                display(contact_sheet)
            else:
                print("But there were no faces in that file!")

print("=== File: small_img.zip ===")
init("readonly/small_img.zip")
print("=== Search: Christopher ===")
search("Christopher")

=== File: small_img.zip ===
=== Search: Christopher ===
